In [3]:
from urllib.request import urlopen
import json
import sqlite3
import time
from datetime import datetime

In [4]:
con = sqlite3.connect("ysk_2019_belediye_secimler.db")
cur = con.cursor()

In [ ]:
#2019 Belediye Seçimleri
il_list  = "https://sonuc.ysk.gov.tr/api/getIlList?secimId=49302&secimTuru=2&sandikTuru=-1&yurtIciDisi=1"
ilcelist = "https://sonuc.ysk.gov.tr/api/getIlceList?secimId=49302&secimTuru=2&ilId={}&secimCevresiId=0&sandikTuru=-1&yurtIciDisi=1"

In [ ]:
for ilid in range(7,82):
    get_url = ilcelist.format(ilid)

    print(ilid)
    response = urlopen(get_url)
    data = json.loads(response.read())
    for item in data:
        try:
            cur.execute("INSERT INTO ilce VALUES(:id,:birim_ID,:ilce_ID,:ilce_ADI,:belde_ID,:secim_CEVRESI_ID,:yerlesim_YERI_TURU,:secilecek_ADAY_SAYISI,:il_ID,:il_ADI,:secim_DETAY_ID)", item)
        except:
            print("NOT INSERTED")
        con.commit()

In [ ]:
con.close()

In [ ]:
muhtarlik_url_ = "https://sonuc.ysk.gov.tr/api/getMuhtarlikList?secimId=49302&secimTuru=2&ilceId={}&beldeId=0&birimId=0&secimCevresiId={}&sandikTuru=-1&yurtIciDisi=1"

In [ ]:
res = cur.execute("""SELECT ilce_ID,secim_CEVRESI_ID FROM ilce""")
ilce_secim = res.fetchall()
for ilceid, secimcevresiid in ilce_secim:
    print(ilceid, secimcevresiid)

In [ ]:
for ilceid, secimcevresiid in ilce_secim:
    get_url = muhtarlik_url_.format(ilceid, secimcevresiid)
    print(ilceid, secimcevresiid)
    response = urlopen(get_url)
    data = json.loads(response.read())
    for item in data:
        try:
            cur.execute("INSERT INTO muhtarlik VALUES (:muhtarlik_ID,:min_SANDIK_NO,:max_SANDIK_NO,:muhtarlik_ADI,:cezaevi_ID,:id,:birim_ID,:ilce_ID,:ilce_ADI,:belde_ID,:secim_CEVRESI_ID,:yerlesim_YERI_TURU,:secilecek_ADAY_SAYISI,:il_ID,:il_ADI,:secim_DETAY_ID)", item)
        except:
            print("NOT INSERTED")
        con.commit()


In [ ]:
secim_sonuc_baslik_url_ = "https://sonuc.ysk.gov.tr/api/getSandikSecimSonucBaslikList?secimId=49302&secimCevresiId={}&ilId=1&bagimsiz=1&secimTuru=2&yurtIciDisi=1"
res = cur.execute("""SELECT ilce_ID,secim_CEVRESI_ID FROM ilce""")
ilce_secim = res.fetchall()
total = len(ilce_secim)
order = 0
print(total)

In [ ]:
for ilceid, secimcevresiid in ilce_secim:
    order += 1
    get_url = secim_sonuc_baslik_url_.format(secimcevresiid)
    print(secimcevresiid, order, total)
    response = urlopen(get_url)
    data = json.loads(response.read())
    for item in data:
        item["secim_CEVRESI_ID"] = secimcevresiid
        print(item)
        try:
            cur.execute("INSERT INTO secim_sonuc_baslik VALUES (:secim_CEVRESI_ID, :sira_NO,:ad,:column_NAME)", item)
        except Exception as e:
            print("NOT INSERTED", e)
        con.commit()

In [ ]:
con.close()

In [ ]:
getSecimSonucList = "https://sonuc.ysk.gov.tr/api/getSecimSonucList?secimId=49302&secimTuru=2&ilId={}&ilceId={}&beldeId={}&birimId=0&muhtarlikId={}&cezaeviId={}&sandikTuru=&sandikNoIlk={}&sandikNoSon={}&ulkeId=&disTemsilcilikId=&gumrukId=&yurtIciDisi=1&sandikRumuzIlk=&sandikRumuzSon=&secimCevresiId={}&sandikId="

In [ ]:
res = cur.execute("""SELECT * FROM muhtarlik WHERE completed is NULL or completed = 0""")
headers = tuple([i[0] for i in res.description])
muhtarlik = res.fetchall()
total = len(muhtarlik)
order = 0
print(total)

In [ ]:
for dd in muhtarlik:
    order += 1
    ddd = dict(zip(headers,dd))
    ilId             = ddd["il_ID"]
    ilceId           = ddd["ilce_ID"]
    beldeId          = ddd["belde_ID"]
    muhtarlikId      = ddd["muhtarlik_ID"]
    cezaeviId        = ddd["cezaevi_ID"]
    sandikNoIlk      = ddd["min_SANDIK_NO"]
    sandikNoSon      = ddd["max_SANDIK_NO"]
    secimCevresiId   = ddd["secim_CEVRESI_ID"]
    get_url = getSecimSonucList.format(ilId, ilceId, beldeId, muhtarlikId, cezaeviId, sandikNoIlk, sandikNoSon, secimCevresiId)
    response = urlopen(get_url)
    data = json.loads(response.read())
    print(ilId,ilId,beldeId, muhtarlikId, order, total)  
    for item in data:
        item["ilce_ID"] = ddd["ilce_ID"]
        item["muhtarlik_ID"] = ddd["muhtarlik_ID"]
        item["min_SANDIK_NO"] = ddd["min_SANDIK_NO"]
        item["max_SANDIK_NO"] = ddd["max_SANDIK_NO"]
        try:
            cur.execute("""UPDATE muhtarlik SET completed = 1, completed_date = '{}'
            WHERE ilce_ID={} AND 
                muhtarlik_ID={} AND 
                min_SANDIK_NO={} AND 
                max_SANDIK_NO={}""".format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),
                                           item["ilce_ID"],
                                           item["muhtarlik_ID"],
                                           item["min_SANDIK_NO"],
                                           item["max_SANDIK_NO"]))
            cur.execute("""INSERT INTO secim_sonuc_ozet VALUES
            (:ilce_ID,
            :muhtarlik_ID,
            :min_SANDIK_NO,
            :max_SANDIK_NO,
            :secilecek_ADAY_SAYISI,
            :birlestirme_TUTANAGI_ILCE,
            :birlestirme_TUTANAGI_KURUL,
            :birlestirme_TUTANAGI_ULKE,
            :birlestirme_TUTANAGI_DIS_TEMSILCILIK,
            :birlestirme_TUTANAGI_GUMRUK_ILCE,
            :birlestirme_TUTANAGI_GUMRUK_KURUL,
            :birlestirme_TUTANAGI_GUMRUK_RUMUZ,
            :birlestirme_TUTANAGI_ULKELER,
            :birlestirme_TUTANAGI_GUMRUKLER,
            :birlestirme_TUTANAGI_TUMDUNYA,
            :acilan_SANDIK_SAYISI,
            :secmen_SAYISI,
            :acilan_SECMEN_SAYISI,
            :oy_KULLANAN_SECMEN_SAYISI,
            :itirazli_GECERLI_OY_SAYISI,
            :itirazsiz_GECERLI_OY_SAYISI,
            :gecerli_OY_TOPLAMI,
            :gecersiz_OY_TOPLAMI,
            :birlestirme_TUTANAGI_IL,
            :toplam_SANDIK_SAYISI)""", item)
            con.commit()
        except Exception as e:
            print("NOT INSERTED", e)
            con.rollback()

In [ ]:
con.close()

In [5]:
getSandikSonucList = "https://sonuc.ysk.gov.tr/api/getSecimSandikSonucList?secimId=49302&secimTuru=2&ilId={}&ilceId={}&beldeId={}&birimId=0&muhtarlikId={}&cezaeviId={}&sandikTuru=&sandikNoIlk={}&sandikNoSon={}&ulkeId=&disTemsilcilikId=&gumrukId=&yurtIciDisi=1&sandikRumuzIlk=&sandikRumuzSon=&secimCevresiId={}&sandikId=&sorguTuru=2"
res = cur.execute("""SELECT * FROM muhtarlik WHERE detay_completed is NULL or detay_completed = 0""")
headers = tuple([i[0] for i in res.description])
secim_sonuc_ozet_list = res.fetchall()
total = len(secim_sonuc_ozet_list)
order = 0
print(total)

32207


In [ ]:
for dd in secim_sonuc_ozet_list:
    order += 1
    ddd = dict(zip(headers,dd))
    ilId             = ddd["il_ID"]
    ilceId           = ddd["ilce_ID"]
    beldeId          = ddd["belde_ID"]
    muhtarlikId      = ddd["muhtarlik_ID"]
    cezaeviId        = ddd["cezaevi_ID"]
    sandikNoIlk      = ddd["min_SANDIK_NO"]
    sandikNoSon      = ddd["max_SANDIK_NO"]
    secimCevresiId   = ddd["secim_CEVRESI_ID"]
    get_url = getSandikSonucList.format(ilId, ilceId, beldeId, muhtarlikId, cezaeviId, sandikNoIlk, sandikNoSon, secimCevresiId)
    response = urlopen(get_url)
    data = json.loads(response.read())
    print(ilId,ilId,beldeId, muhtarlikId, order, total, end="")
    for item in data:
        item["ilce_ID"] = ddd["ilce_ID"]
        item["muhtarlik_ID"] = ddd["muhtarlik_ID"]
        item["min_SANDIK_NO"] = ddd["min_SANDIK_NO"]
        item["max_SANDIK_NO"] = ddd["max_SANDIK_NO"]
        try:
            cur.execute("""UPDATE muhtarlik SET detay_completed = 1, detay_completed_date = '{}'
            WHERE ilce_ID={} AND 
                muhtarlik_ID={} AND 
                min_SANDIK_NO={} AND 
                max_SANDIK_NO={}""".format(datetime.today().strftime('%Y-%m-%d %H:%M:%S'),
                                           item["ilce_ID"],
                                           item["muhtarlik_ID"],
                                           item["min_SANDIK_NO"],
                                           item["max_SANDIK_NO"]))
            cur.execute("""INSERT INTO sandik_sonuc VALUES
            (:muhtarlik_ID,
            :birim_ID,
            :ilce_ID,
            :ulke_ID,
            :sandik_NO,
            :sandik_SONUC_TUTANAGI,
            :sandik_DOKUM_CETVELI,
            :denetim_TUTANAGI,
            :ilce_ADI,
            :muhtarlik_ADI,
            :cezaevi_ID,
            :dis_TEMSILCILIK_ID,
            :il_ID,
            :birlestirme_TUTANAGI_ILCE,
            :birlestirme_TUTANAGI_KURUL,
            :birlestirme_TUTANAGI_ULKE,
            :birlestirme_TUTANAGI_DIS_TEMSILCILIK,
            :birlestirme_TUTANAGI_GUMRUK_ILCE,
            :birlestirme_TUTANAGI_GUMRUK_KURUL,
            :birlestirme_TUTANAGI_GUMRUK_RUMUZ,
            :birlestirme_TUTANAGI_ULKELER,
            :birlestirme_TUTANAGI_GUMRUKLER,
            :birlestirme_TUTANAGI_TUMDUNYA,
            :birlestirme_TUTANAGI_IL,
            :il_ADI,
            :bagimsiz_TOPLAM_OY,
            :gumruk_ID,
            :sandik_RUMUZ,
            :gumruk_SANDIK_TARIHI,
            :belde_ID,
            :secim_TURU,
            :parti28_ALDIGI_OY,
            :parti29_ALDIGI_OY,
            :parti30_ALDIGI_OY,
            :bagimsiz1_ALDIGI_OY,
            :bagimsiz2_ALDIGI_OY,
            :bagimsiz3_ALDIGI_OY,
            :bagimsiz4_ALDIGI_OY,
            :bagimsiz5_ALDIGI_OY,
            :bagimsiz6_ALDIGI_OY,
            :bagimsiz7_ALDIGI_OY,
            :bagimsiz8_ALDIGI_OY,
            :bagimsiz9_ALDIGI_OY,
            :bagimsiz10_ALDIGI_OY,
            :bagimsiz11_ALDIGI_OY,
            :bagimsiz12_ALDIGI_OY,
            :bagimsiz13_ALDIGI_OY,
            :bagimsiz14_ALDIGI_OY,
            :bagimsiz15_ALDIGI_OY,
            :bagimsiz16_ALDIGI_OY,
            :bagimsiz17_ALDIGI_OY,
            :bagimsiz18_ALDIGI_OY,
            :bagimsiz19_ALDIGI_OY,
            :bagimsiz20_ALDIGI_OY,
            :bagimsiz21_ALDIGI_OY,
            :bagimsiz22_ALDIGI_OY,
            :bagimsiz23_ALDIGI_OY,
            :bagimsiz24_ALDIGI_OY,
            :bagimsiz25_ALDIGI_OY,
            :bagimsiz26_ALDIGI_OY,
            :bagimsiz27_ALDIGI_OY,
            :bagimsiz28_ALDIGI_OY,
            :bagimsiz29_ALDIGI_OY,
            :bagimsiz30_ALDIGI_OY,
            :bagimsiz31_ALDIGI_OY,
            :bagimsiz32_ALDIGI_OY,
            :bagimsiz33_ALDIGI_OY,
            :bagimsiz34_ALDIGI_OY,
            :bagimsiz35_ALDIGI_OY,
            :bagimsiz36_ALDIGI_OY,
            :bagimsiz37_ALDIGI_OY,
            :bagimsiz38_ALDIGI_OY,
            :bagimsiz39_ALDIGI_OY,
            :bagimsiz40_ALDIGI_OY,
            :secim_ID,
            :sandik_ID,
            :secmen_SAYISI,
            :oy_KULLANAN_SECMEN_SAYISI,
            :itirazli_GECERLI_OY_SAYISI,
            :itirazsiz_GECERLI_OY_SAYISI,
            :gecerli_OY_TOPLAMI,
            :gecersiz_OY_TOPLAMI,
            :bagimsiz41_ALDIGI_OY,
            :bagimsiz42_ALDIGI_OY,
            :bagimsiz43_ALDIGI_OY,
            :bagimsiz44_ALDIGI_OY,
            :bagimsiz45_ALDIGI_OY,
            :bagimsiz46_ALDIGI_OY,
            :bagimsiz47_ALDIGI_OY,
            :bagimsiz48_ALDIGI_OY,
            :bagimsiz49_ALDIGI_OY,
            :bagimsiz50_ALDIGI_OY,
            :ittifak1_ALDIGI_OY,
            :ittifak2_ALDIGI_OY,
            :ittifak3_ALDIGI_OY,
            :ittifak4_ALDIGI_OY,
            :ittifak5_ALDIGI_OY,
            :versiyon,
            :son_ISLEM_TARIHI,
            :degisiklik_NEDENI,
            :sandik_SONUC_ID,
            :parti1_ALDIGI_OY,
            :parti2_ALDIGI_OY,
            :parti3_ALDIGI_OY,
            :parti4_ALDIGI_OY,
            :parti5_ALDIGI_OY,
            :parti6_ALDIGI_OY,
            :parti7_ALDIGI_OY,
            :parti8_ALDIGI_OY,
            :parti9_ALDIGI_OY,
            :parti10_ALDIGI_OY,
            :parti11_ALDIGI_OY,
            :parti12_ALDIGI_OY,
            :parti13_ALDIGI_OY,
            :parti14_ALDIGI_OY,
            :parti15_ALDIGI_OY,
            :parti16_ALDIGI_OY,
            :parti17_ALDIGI_OY,
            :parti18_ALDIGI_OY,
            :parti19_ALDIGI_OY,
            :parti20_ALDIGI_OY,
            :parti21_ALDIGI_OY,
            :parti22_ALDIGI_OY,
            :parti23_ALDIGI_OY,
            :parti24_ALDIGI_OY,
            :parti25_ALDIGI_OY,
            :parti26_ALDIGI_OY,
            :parti27_ALDIGI_OY)""", item)
            con.commit()
        except Exception as e:
            print("NOT INSERTED", e)
            con.rollback()

In [7]:
con.close()